In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from PIL import Image
import csv
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import *
import time
import copy
import os
import tensorflow as tf
from skimage.transform import resize
import imageio

In [2]:
root = "/Users/rushi/Downloads/SoML-50/data/"
annotations = "/Users/rushi/Downloads/SoML-50/annotations.csv"


In [3]:
class ShauryaDataset(Dataset):
    def __init__(self,root,annotations,transform=ToTensor()):
        self.root = root
        self.transform = transform
        rows = []
        with open(annotations,mode='r') as csvfile:
            csvreader = csv.reader(csvfile)
            fields = next(csvreader)
            for row in csvreader:
                rows.append(row)
        self.data = rows
        

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.root + self.data[idx][0]
        image = Image.open(image_path)
        image = ToTensor()(image).squeeze(0)
        image = resize(image,(28,28*3),mode = 'constant')
        img1 = image[:,:28]
        img2 = image[:,28:56]
        img3 = image[:,56:]
        img1 = img1.reshape(28*28)
        img2 = img2.reshape(28*28)
        img3 = img3.reshape(28*28)
        temp = self.data[idx][1]
        #y = (int)(self.data[idx][2])+9
        if(temp == "prefix"):
            y = 0
            return image,img1,img2,img3,y
        elif(temp == "infix"):
            y = 1
            return image,img2,img1,img3,y
        else:
            y = 2
            return image,img3,img2,img1,y

In [4]:
dataset = ShauryaDataset(root,annotations)

In [5]:
from torch.utils.data import random_split
dataset_train, dataset_val, dataset_test = random_split(dataset,[40000,5000,5000])

In [6]:
batchsize = 100
dataset_new = DataLoader(dataset,len(dataset),shuffle = False)
print(len(dataset_new))
train_loader = DataLoader(dataset_test,batchsize,shuffle=False)
val_loader = DataLoader(dataset_val,batchsize,shuffle=False)
inputsize = 28*84
classes = 3


1


In [7]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [8]:
class Net_LABELS(nn.Module):   
    def __init__(self):
        super(Net_LABELS, self).__init__()
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(1, 4, kernel_size=5, stride=1, padding=1),
            nn.BatchNorm2d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear = nn.Sequential(
            nn.Linear(4*6*20,classes),
            nn.ReLU(),
        )


    # Defining the forward pass    
    def forward(self, x):
        x = x.reshape(-1,1,28,84)
        x = self.cnn_layers(x)
        x = x.reshape(-1,4*6*20)
        x = self.linear(x)
        return x

    def training_step(self, batch):
        images,img1,img2,img3, labels = batch
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images,img1,img2,img3, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss, 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))
    

model = Net_LABELS()

In [9]:
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    optimizer = opt_func(model.parameters(), lr)
    history = []

    for epoch in range(epochs):

        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)

    return history

In [10]:
fit(100,0.01,model,train_loader,val_loader)
torch.save(model.state_dict(),'Last_hope_labels.pt')

/Users/rushi/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch [0], val_loss: 0.5279, val_acc: 0.8170
Epoch [1], val_loss: 0.3765, val_acc: 0.8678
Epoch [2], val_loss: 0.2980, val_acc: 0.8980
Epoch [3], val_loss: 0.2509, val_acc: 0.9152
Epoch [4], val_loss: 0.2174, val_acc: 0.9240
Epoch [5], val_loss: 0.1918, val_acc: 0.9346
Epoch [6], val_loss: 0.1717, val_acc: 0.9428
Epoch [7], val_loss: 0.1559, val_acc: 0.9462
Epoch [8], val_loss: 0.1443, val_acc: 0.9494
Epoch [9], val_loss: 0.1352, val_acc: 0.9518
Epoch [10], val_loss: 0.1280, val_acc: 0.9538
Epoch [11], val_loss: 0.1221, val_acc: 0.9546
Epoch [12], val_loss: 0.1170, val_acc: 0.9572
Epoch [13], val_loss: 0.1126, val_acc: 0.9586
Epoch [14], val_loss: 0.1088, val_acc: 0.9614
Epoch [15], val_loss: 0.1054, val_acc: 0.9626
Epoch [16], val_loss: 0.1022, val_acc: 0.9630
Epoch [17], val_loss: 0.0994, val_acc: 0.9638
Epoch [18], val_loss: 0.0967, val_acc: 0.9658
Epoch [19], val_loss: 0.0943, val_acc: 0.9668
Epoch [20], val_loss: 0.0920, val_acc: 0.9680
Epoch [21], val_loss: 0.0899, val_acc: 0.968